## Setup

In [ ]:
!git clone https://github.com/XavierSpycy/NumPyMultilayerPerceptron.git
%cd NumPyMultilayerPerceptron

## Toy Example

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nn.mlp import MultilayerPerceptron
from nn.layers import Dense

In [ ]:
np.random.seed(3407)
class_1 = np.hstack([np.random.normal( 1, 1, size=(500, 2)),  np.ones(shape=(500, 1))])
class_2 = np.hstack([np.random.normal(-1, 1, size=(200, 2)), -np.ones(shape=(200, 1))])
dataset = np.vstack([class_1, class_2])
X, y = dataset[:,:2], dataset[:,2]

plt.figure(figsize=(6, 6))
plt.scatter(class_1[:,0], class_1[:,1], label='1')
plt.scatter(class_2[:,0], class_2[:,1], label='-1')
plt.grid()
plt.legend()
plt.show()

In [ ]:
layers = [
    Dense(2, 4, activation='leaky_relu', init='kaiming_normal', init_params={'mode': 'out'}),
    Dense(4, 3, activation='hardswish', init='xavier_normal'),
    Dense(3, 2, activation='relu', init='kaiming_normal', init_params={'mode': 'in'}),
    Dense(2, 1, activation='tanh', init='xavier_uniform')
]
mlp = MultilayerPerceptron(layers)
mlp.compile(optimizer='Adam',
            loss='MeanSquareError')
mlp.fit(X, y, epochs=80, batch_size=8, use_progress_bar=True)
mlp.plot_loss()

In [ ]:
xx, yy = np.meshgrid(np.arange(-2, 2, .02),np.arange(-2, 2, .02))
Z = mlp.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
plt.pcolormesh(xx, yy, Z>0, cmap='cool')
plt.scatter(X[:,0], X[:,1], c=[['b', 'r'][int(d>0)] for d in y], s=100)
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.grid()
plt.title('Targets')
plt.subplot(1,2,2)
plt.pcolormesh(xx, yy, Z>0, cmap='cool')
plt.scatter(X[:,0], X[:,1], c=[['b', 'r'][int(d>0)] for d in mlp.predict(X)], s=100)
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.grid()
plt.title('Predictions')

## Pipeline

In [ ]:
import numpy as np
from datasets.data_loader import datasets
from nn.mlp import MultilayerPerceptron
from nn.layers import Dense, Dropout
from nn.optim import Adam
from nn.schedules import MultiStepLR
from sklearn.metrics import accuracy_score as accuracy

X_train, y_train = datasets(train=True)
X_test, y_test = datasets(train=False)

np.random.seed(3407)

layers = [
    Dense(128, 120, activation='elu', init='kaiming_uniform'),
    Dropout(dropout_rate=0.25),
    Dense(120, 112, activation='elu', init='kaiming_uniform'),
    Dropout(dropout_rate=0.20),
    Dense(112, 96, activation='elu', init='kaiming_uniform'),
    Dropout(dropout_rate=0.15),
    Dense(96, 64, activation='elu', init='kaiming_uniform'),
    Dropout(dropout_rate=0.10),
    Dense(64, 48, activation='elu', init='kaiming_uniform'),
    Dropout(dropout_rate=0.05),
    Dense(48, 32, activation='elu', init='kaiming_uniform'),
    Dense(32, 24, activation='elu', init='kaiming_uniform'),
    Dense(24, 16, activation='elu', init='kaiming_uniform'),
    Dense(16, 10, activation='softmax', init='xavier_uniform')
]

mlp = MultilayerPerceptron(layers)
optimizer = Adam(lr=1e-3, weight_decay=0.02)
scheduler = MultiStepLR(optimizer, milestones=[100, 200, 300, 400], gamma=0.8)
mlp.compile(optimizer=optimizer,
            loss='CrossEntropy',
            scheduler=scheduler)
mlp.fit(X_train, y_train, epochs=500, batch_size=32, use_progress_bar=True)
loss = mlp.loss_tracker()
train_time = mlp.training_time()
print(f'Training time: {train_time:.2f} second(s).')
print(f'Loss: {loss[-1]:.2f}.')

print(f"Accuracy on the training set is: {accuracy(y_train, mlp.predict(X_train)):.2%}." )
print(f"Accuracy on the test set is: {accuracy(y_test, mlp.predict(X_test)):.2%}.")

## Colab Built-in Datasets

In [ ]:
# Classification task
import numpy as np
import pandas as pd
from nn.layers import Dense
from nn.optim import Adam
from nn.mlp import MultilayerPerceptron
from sklearn.metrics import accuracy_score as accuracy
data_train = pd.read_csv('/content/sample_data/mnist_train_small.csv', header=None).to_numpy()
data_test = pd.read_csv('/content/sample_data/mnist_test.csv', header=None).to_numpy()

X_train, y_train = data_train[:,1:], data_train[:,0]
X_test, y_test = data_test[:,1:], data_test[:,0]
X_train = X_train / 255
X_test = X_test / 255

np.random.seed(3407)

layers = [
    Dense(784, 128, activation='relu', init='kaiming_uniform'),
    Dense(128, 16, activation='relu', init='kaiming_uniform'),
    Dense(16, 10, activation='softmax')
]

mlp = MultilayerPerceptron(layers)
mlp.compile(optimizer=Adam(),
            loss='CrossEntropy')
mlp.fit(X_train, y_train, epochs=10, batch_size=32, use_progress_bar=True)
loss = mlp.loss_tracker()
train_time = mlp.training_time()
print(f'Training time: {train_time:.2f} second(s).')
print(f'Loss: {loss[-1]:.2f}.')
mlp.plot_loss()

print(f"Accuracy on the training set is: {accuracy(y_train, mlp.predict(X_train)):.2%}." )
print(f"Accuracy on the test set is: {accuracy(y_test, mlp.predict(X_test)):.2%}.")

In [ ]:
# Regression task
import numpy as np
import pandas as pd
from nn.layers import Dense
from nn.optim import Adam
from nn.mlp import MultilayerPerceptron
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
data_train = pd.read_csv('/content/sample_data/california_housing_train.csv').to_numpy()
data_test = pd.read_csv('/content/sample_data/california_housing_test.csv').to_numpy()
X_train, y_train = data_train[:, :-1], data_train[:, -1]
X_test, y_test = data_test[:, :-1], data_test[:, -1]

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

np.random.seed(3407)

layers = [
    Dense(8, 6, activation='relu', init='kaiming_uniform'),
    Dense(6, 4, activation='relu', init='kaiming_uniform'),
    Dense(4, 2, activation='relu', init='kaiming_uniform'),
    Dense(2, 1)
]

mlp = MultilayerPerceptron(layers)
mlp.compile(optimizer=Adam(),
            loss='MeanSquareError')

mlp.fit(X_train, y_train, epochs=500, batch_size=32, use_progress_bar=True)
train_time = mlp.training_time()
print(f'Training time: {train_time:.2f} second(s).')
mlp.plot_loss()

print(f"R^2 on the training set is: {r2_score(y_train, mlp.predict(X_train)):.2%}")
print(f"R^2 on the test set is: {r2_score(y_test, mlp.predict(X_test)):.2%}")